For demo purposes, the number of runs per model (N) should be reduced to 1, in which case this benchmark would take around 2 hours to complete on a V100 GPU.  
If you modify N to be 5 as was done in the paper, you should expect around 10 hours of run time to complete all testing.  

In [ ]:
# Number of runs per model
n_runs = 5

# Set random seeds for reproducibility
import torch
torch.manual_seed(0)
import numpy as np
np.random.seed(0)
import random
random.seed(0)

# Colab does not like downloading large files directly
# If you wish to save the results
# Consider mounting Google Drive to save the results
#   also uncomment the lines in the last code block
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Obtaining EEGEyeNet's benchmark framework
!git clone https://github.com/ardkastrati/EEGEyeNet.git
# Obtaining our model implementations (our model and EEGViT)
!git clone https://github.com/AmCh-Q/CSCI6907Project.git
# Copy our model implementations to EEGEyeNet
!cp -l CSCI6907Project/our_models.py EEGEyeNet
!cp -l CSCI6907Project/EEGViT_pretrained.py EEGEyeNet

Cloning into 'EEGEyeNet'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 302 (delta 104), reused 88 (delta 88), pack-reused 180
Receiving objects: 100% (302/302), 84.92 KiB | 5.31 MiB/s, done.
Resolving deltas: 100% (165/165), done.
Cloning into 'CSCI6907Project'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 132 (delta 32), reused 53 (delta 29), pack-reused 64
Receiving objects: 100% (132/132), 1.93 MiB | 26.31 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
%cd EEGEyeNet

/content/EEGEyeNet


In [ ]:
# Obtaining the datasets
!mkdir ./data

#!wget -O ./data/Position_task_with_dots_synchronised_min.npz https://osf.io/download/ge87t/
!cp /content/drive/MyDrive/Class/CSCI6907/Position_task_with_dots_synchronised_min.npz ./data

#!wget -O ./data/Position_task_with_dots_synchronised_min_hilbert.npz https://osf.io/download/bmrn9/
!cp /content/drive/MyDrive/Class/CSCI6907/Position_task_with_dots_synchronised_min_hilbert.npz ./data

In [ ]:
# Configuring task, and dataset
# Modified from https://github.com/ardkastrati/EEGEyeNet
from config import config, build_file_name

config['task'] = 'Position_task'
config['dataset'] = 'dots'
config['preprocessing'] = 'min'
config['feature_extraction'] = False

In [ ]:
# Google Colab does not like tee-redirecting stdout (nothing gets printed)
# Colab also initialized its own logging configuration that must be overwritten
# So EEGEyeNet's main() has been modified for Colab
import sys
import time
import logging
import numpy as np
from config import create_folder
from utils import IOHelper
import benchmark
from hyperparameters import merge_models

# Setting up logging
create_folder()
logging.basicConfig(filename=config['info_log'], level=logging.INFO, force=True)
logging.info('Started the Logging')
logging.info(f"Using {config['framework']}")
start_time = time.time()

# RMSE Scoring
from sklearn.metrics import mean_squared_error
scoring = (lambda y, y_pred : np.sqrt(mean_squared_error(y, y_pred)))

(1) EEGEyeNet's ML models only works on hilbert transform feature-extracted data.  
(2) EEGEyeNet's DL models and (3) EEGViT and our model only works on raw, non-feature-extracted data.  
That means they cannot be run simultaneously.  
We have opted to split the models into three groups, and train and test them sparately in the following four code blocks.  
If you only wish to test our model, you can comment out the next 2 code blocks, and comment out the definition of EEGViT in the 3rd next code block.

In [ ]:
# Training and benchmarking the EEGEyeNet ML models
# Modified from https://github.com/ardkastrati/EEGEyeNet
config['feature_extraction'] = True
from hyperparameters import our_ML_dummy_models, our_ML_models

#Load the data
IOHelper.config['all_EEG_file'] = "Position_task_with_dots_synchronised_min_hilbert.npz"
trainX, trainY = IOHelper.get_npz_data(config['data_dir'], verbose=True)

# Start benchmark
np.savetxt(config['model_dir']+'/config_EEGEyeNet_ML.csv', [config['task'], config['dataset'], config['preprocessing']], fmt='%s')
models = merge_models(our_ML_dummy_models, our_ML_models)
models = models[config['task']][config['dataset']][config['preprocessing']]
benchmark.try_models(
    trainX=trainX,
    trainY=trainY[:,1:],
    ids=trainY[:,0],
    models=models,
    N=n_runs,
    scoring=scoring,
    save_trail='_EEGEyeNet_ML_models')

# Clean up
del trainX, trainY

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider

In [ ]:
# Training and benchmarking the EEGEyeNet DL models
# Modified from https://github.com/ardkastrati/EEGEyeNet
config['feature_extraction'] = False
from hyperparameters import our_DL_models

# Load the data
IOHelper.config['all_EEG_file'] = "Position_task_with_dots_synchronised_min.npz"
trainX, trainY = IOHelper.get_npz_data(config['data_dir'], verbose=True)

# Start benchmark
np.savetxt(config['model_dir']+'/config_EEGEyeNet_DL.csv', [config['task'], config['dataset'], config['preprocessing']], fmt='%s')
models = our_DL_models
models = models[config['task']][config['dataset']][config['preprocessing']]
benchmark.try_models(
    trainX=trainX,
    trainY=trainY[:,1:],
    ids=trainY[:,0],
    models=models,
    N=n_runs,
    scoring=scoring,
    save_trail='_EEGEyeNet_DL_models')

# Clean up
del trainX, trainY

In [ ]:
# Add our model and SOTA
from EEGViT_pretrained import EEGViT_Pretrained
from our_models import Ours_Pretrained
EEGViT_and_our_models = {
    'Position_task': {
        'dots' : {
            'min' : {
                'EEGViT': [EEGViT_Pretrained, {
                    'model_name': "EEGViT",
                    'nb_models': 1,
                    'batch_size': 64,
                    'n_epoch': 15,
                    'learning_rate': 1e-4,
                    'vit_model_name': "google/vit-base-patch16-224"}],
                'Ours_Pretrained': [Ours_Pretrained, {
                    'model_name': "Ours_Pretrained",
                    'nb_models': 1,
                    'batch_size': 64,
                    'n_epoch': 15,
                    'learning_rate': 1e-4,
                    'vit_model_name': "google/vit-base-patch16-224"}]
            }
        }
    }
}

In [ ]:
# Training and benchmarking EEGViT-pretrained model and our model
config['feature_extraction'] = False

# Load the data
IOHelper.config['all_EEG_file'] = "Position_task_with_dots_synchronised_min.npz"
trainX, trainY = IOHelper.get_npz_data(config['data_dir'], verbose=True)

# Start benchmark
np.savetxt(config['model_dir']+'/config_EEGViT_and_ours.csv', [config['task'], config['dataset'], config['preprocessing']], fmt='%s')
models = EEGViT_and_our_models[config['task']][config['dataset']][config['preprocessing']]
benchmark.try_models(
    trainX=trainX,
    trainY=trainY[:,1:],
    ids=trainY[:,0],
    models=models,
    N=n_runs,
    scoring=scoring,
    save_trail='_EEGViT_and_our_models')

# Clean up
del trainX, trainY

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- vit.embeddings.patch_embeddings.projection.weight: found shape torch.Size([768, 3, 16, 16]) in the checkpoint and torch.Size([768, 256, 8, 1]) in the model instantiated
- vit.embeddings.position_embeddings: found shape torch.Size([1, 197, 768]) in the checkpoint and torch.Size([1, 225, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Training Loss: 75641.72378656914
Epoch 1, Val Loss: 27474.184814453125
Epoch 2, Training Loss: 24981.859375
Epoch 2, Val Loss: 23504.90380859375
Epoch 3, Training Loss: 22152.9711768617
Epoch 3, Val Loss: 20404.342407226562
Epoch 4, Training Loss: 17840.923927859043
Epoch 4, Val Loss: 16688.5688273112
Epoch 5, Training Loss: 15440.270457945478
Epoch 5, Val Loss: 16192.830546061197
Epoch 6, Training Loss: 14596.640936668882
Epoch 6, Val Loss: 16278.141296386719
Epoch 7, Training Loss: 13521.819830452128
Epoch 7, Val Loss: 15029.24180094401
Epoch 8, Training Loss: 13063.561762383644
Epoch 8, Val Loss: 14943.929646809896
Epoch 9, Training Loss: 12680.708331948139
Epoch 9, Val Loss: 14888.913208007812
Epoch 10, Training Loss: 12293.799187583112
Epoch 10, Val Loss: 14847.694742838541


In [ ]:
# Ending Logs
logging.info("--- Runtime: %s seconds ---" % (time.time() - start_time))
logging.info('Finished Logging')

The results of the models can be found in directory ./runs  
Note that the scores are in pixels, where 2 pixels = 1 millimeter

In [ ]:
# You can zip the files of the run(s) if you want to save it
!zip -r runs.zip ./runs

# However, it is difficult to download massive files directly from Colab
# The best solution for now is to copy it to your Google Drive
!cp runs.zip /content/drive/MyDrive